In [0]:
CREATE OR REFRESH MATERIALIZED VIEW hourly_agg_created_with_enzyme
 AS 
SELECT 
  -- Time dimension
  date_trunc('hour', event_timestamp) AS event_hour_timestamp,
  
  -- Site grouping
  site_id,
  
  -- Tracker performance metrics
  COUNT(*) AS total_readings,
  COUNT(DISTINCT tracker_row) AS active_tracker_rows,
  
  -- Angle tracking performance
  AVG(angle_actual) AS avg_angle_actual,
  AVG(angle_target) AS avg_angle_target,
  AVG(ABS(angle_actual - angle_target)) AS avg_angle_deviation,
  MAX(ABS(angle_actual - angle_target)) AS max_angle_deviation,
  STDDEV(angle_actual - angle_target) AS angle_tracking_variance,
  
  -- Additional angle deviation metric
  AVG(angle_dev) AS avg_angle_dev,
  MAX(angle_dev) AS max_angle_dev,
  
  -- Motor performance
  AVG(rpm) AS avg_motor_rpm,
  MAX(rpm) AS max_motor_rpm,
  MIN(rpm) AS min_motor_rpm,
  AVG(torque) AS avg_motor_torque,
  MAX(torque) AS max_motor_torque,
  
  -- Temperature monitoring
  AVG(motor_temp) AS avg_motor_temp,
  MAX(motor_temp) AS max_motor_temp,
  AVG(ambient_temp) AS avg_ambient_temp,
  MAX(ambient_temp) AS max_ambient_temp,
  MIN(ambient_temp) AS min_ambient_temp,
  
  -- Environmental conditions
  AVG(wind_speed) AS avg_wind_speed,
  MAX(wind_speed) AS max_wind_speed,
  AVG(irradiance) AS avg_irradiance,
  MAX(irradiance) AS max_irradiance,
  SUM(irradiance) / COUNT(*) AS weighted_avg_irradiance,
  
  -- Energy estimation (proxy)
  AVG(kwh_proxy) AS avg_kwh_proxy,
  SUM(kwh_proxy) AS total_kwh_proxy,
  
  -- System health indicators
  COUNT(CASE WHEN fault_code IS NOT NULL AND fault_code != '' THEN 1 END) AS fault_count,
  COUNT(CASE WHEN fault_code IS NULL OR fault_code = '' THEN 1 END) AS healthy_readings,
  ROUND(
    (COUNT(CASE WHEN fault_code IS NULL OR fault_code = '' THEN 1 END) * 100.0) / COUNT(*), 
    2
  ) AS uptime_percentage,
  
  -- Fault analysis
  collect_set(fault_code) AS unique_fault_codes,
  
  -- Tracking efficiency metrics
  AVG(CASE 
    WHEN ABS(angle_actual - angle_target) <= 1 THEN 100.0
    WHEN ABS(angle_actual - angle_target) <= 2 THEN 95.0
    WHEN ABS(angle_actual - angle_target) <= 5 THEN 85.0
    ELSE 70.0
  END) AS tracking_efficiency_score,
  
  -- Weather impact indicators
  AVG(CASE WHEN wind_speed > 10 THEN 1 ELSE 0 END) * 100 AS high_wind_percentage,
  AVG(CASE WHEN irradiance > 800 THEN 1 ELSE 0 END) * 100 AS high_irradiance_percentage,
  
  -- Data quality metrics
  COUNT(CASE WHEN _rescued_data IS NOT NULL THEN 1 END) AS rescued_data_count,
  ROUND(
    (COUNT(CASE WHEN _rescued_data IS NULL THEN 1 END) * 100.0) / COUNT(*), 
    2
  ) AS data_quality_percentage

FROM soni.default.nextracker_raw_stream
WHERE event_timestamp IS NOT NULL
GROUP BY all